In [1]:
import sqlite3
import pandas as pd
import numpy as np
import pymysql
import re
from kss import kss
from konlpy.tag import Mecab
import pickle
import unicodedata

In [2]:
mecab = Mecab()
tmp_ref_dic = pd.read_csv('ref_dic.csv')#ref_dic.reset_index(drop=True)
tmp_pos_set = tmp_ref_dic['명칭'].values.tolist()
pos_sel_no = []
for i in range(0, len(tmp_pos_set)):
    try :
        if unicodedata.name(tmp_pos_set[i][0]).split(' ')[0] in ('HANGUL', 'LATIN'):
            if len(tmp_pos_set[i]) > 1 :
                pos_sel_no.append(i)
            elif unicodedata.name(tmp_pos_set[i][0]).split(' ')[0] in ('HANGUL'):
                pos_sel_no.append(i)
    except:
        pass #rint(i)

ref_dic = tmp_ref_dic.iloc[pos_sel_no, :].reset_index(drop=True).copy()
pos_set = [x.strip() for x in ref_dic['명칭'].values.tolist()]

In [3]:
con1 = sqlite3.connect('ensol_news_db.sqlite')
cur1 = con1.cursor()
cur1.execute("select * from mobility")
db_tab = pd.DataFrame(cur1.fetchall())
db_tab = db_tab.iloc[:,[1,2,4,5]].drop_duplicates()

In [4]:
def _call_db_info():
    return pymysql.connect(
        host = 'trend.cb7jqghocrtb.ap-northeast-2.rds.amazonaws.com',
        port= 3306,
        user = 'root',
        password='ensol2020!',
        db = 'trend',
        charset = 'utf8')


conn = _call_db_info()
curs = conn.cursor()
tmp_insert_sql = "select * from content where date >= 20210701"
curs.execute(tmp_insert_sql)        
tmp_rslt = pd.DataFrame(curs.fetchall())
conn.commit()
conn.close()

news_tab = tmp_rslt.iloc[:,1:].drop_duplicates()

In [5]:
# Train
#print(db_tab.shape, news_tab.shape)
#tmp_article= pd.concat((db_tab.iloc[:,3], news_tab.iloc[:,3]))
# Test
print(news_tab.shape)
tmp_article= news_tab.iloc[:,3]

tmp_article = tmp_article.drop_duplicates()
tmp_article = tmp_article.reset_index(drop=True)
print(tmp_article.shape)

(210, 4)
(161,)


In [6]:
def extract_parenthese(str):
    items_lst = re.findall('\(([^)]+)', str) #extracts string in () 
    newList = [x for x in items_lst if len(x)>=2] # more than 2
    return newList

def extract_quotes(str):
    items_lst = re.findall('"([^"]*)"', str)
    return items_lst

def parentheses_(tmp_input_sent):    
    tmp_input_sent = re.sub(pattern='\(+', repl=' ', string=tmp_input_sent)#tmp_input_sent = re.sub(pattern='\(\(', repl='\(', string=tmp_input_sent)
    tmp_input_sent = re.sub(pattern='\)+', repl=' ', string=tmp_input_sent)#tmp_input_sent = re.sub(pattern='\)\)', repl='\)', string=tmp_input_sent)
    tmp_input_sent = re.sub(pattern=' +', repl=' ', string=tmp_input_sent)
    input_sent = re.sub(pattern='\\\\',   repl='', string=tmp_input_sent)
    return input_sent
    '''
    tmp_sent1, tmp_sent2 = [], []    
    s_re = re.compile('\(')#tmp_sentence[25])#.match('\(')
    e_re = re.compile('\)')
    s_m = [(m.start(0), m.end(0)) for m in s_re.finditer(input_sent)]#tmp_sentence[25])]
    e_m = [(m.start(0), m.end(0)) for m in e_re.finditer(input_sent)]#tmp_sentence[25])]
    m = []
    for i in range(0, len(s_m)):
        if s_m[i][1] < e_m[i][0]:
            m.append((s_m[i], e_m[i]))
        else:
            for j in range(i+1, len(e_m)):
                if s_m[i][1] < e_m[j][0]:
                    m.append((s_m[i], e_m[j]))
                    break;                   

    if len(m) > 0:
        for i in range(0, len(m)):
            if i == 0:
                tmp_sent1.append(input_sent[:m[i][0][0]])
            else :
                tmp_sent1.append(input_sent[m[(i-1)][1][1]:m[i][0][0]])                        
            tmp_sent2.append(input_sent[m[i][0][1]:m[i][1][0]])
        tmp_sent1.append(input_sent[m[-1][1][1]:])    
        return ' '.join(tmp_sent1 + tmp_sent2)
    else:
        return input_sent
    '''
    
def pos_select(input_sent, p_set):
    tmp_ = []
    tmp_s_pt = []
    for i,x in enumerate(p_set):
        try :            
            #tmp_s_pt.append(input_sent.index(x))
            tmp_.append([i, input_sent.index(x), len(p_set[i]), 0 if p_set[i].find(' ')== -1 else 1])            
        except:
            pass        
    return tmp_#, tmp_s_pt

In [7]:
step0_ptn= '[\'\‘\’]'
step1_ptn= '[\u00a0\u3000①②③④⑤⑥⑦⑧⑨⑩』◦※→®↑↓‣★▶■△◇◆▲○●\{\}\[\]\/?,+;:‧·ᆞ…》ⓒ|*~`\""“”!^_<>@\#&\\\=\'\n]'     
step2_ptn= '[\.]' 
article_sent = []

article_ner_l1, article_ner_l2 = [], []

for a in range(0, tmp_article.shape[0]):
    tmp_a = re.sub(pattern=step0_ptn, repl='\"', string=tmp_article.iloc[a])#.replace('\'‘’', '\"')
    tmp_a = re.sub(pattern=step1_ptn, repl='', string=tmp_a)
    tmp_sentence = kss.split_sentences(tmp_a)
    tmp_sentence = [re.sub(pattern=step2_ptn, repl='', string=s) for s in tmp_sentence]
    tmp_sentence = [parentheses_(s) for s in tmp_sentence] 

    ner_sent, ner_tag_l1, ner_tag_l2 = [], [], []

    for sent in range(0, len(tmp_sentence)):
        
        tmp_ner = []
        tmp_sent = re.sub(pattern=' +', repl=' ', string=tmp_sentence[sent])
        sent_split = tmp_sent.split(' ')
        tmp_ner_l1 = ['O'] * len(sent_split)
        tmp_ner_l2 = ['O'] * len(sent_split)
        
        if len(pos_select(tmp_sent, pos_set)) == 0:
            #print(a, sent)
            break
        tmp_pos_info = pd.DataFrame(pos_select(tmp_sent, pos_set)).sort_values(by=1).reset_index(drop=True)        

        for i in range(0, tmp_pos_info.shape[0]):
            if tmp_pos_info[3].loc[i] == 0:        
                tmp_ner_pos = len(tmp_sent[:tmp_pos_info[1].loc[i] + tmp_pos_info[2].loc[i] ].split())-1
                tmp_ner_l1[tmp_ner_pos] = ref_dic['대분류'].loc[tmp_pos_info[0].loc[i]]
            else:
                tmp_ner_pos1 = len(tmp_sent[:tmp_pos_info[1].loc[i]]) 
                tmp_ner_pos2 = len(tmp_sent[:tmp_pos_info[1].loc[i] + tmp_pos_info[2].loc[i] ].split())
                for p in range(tmp_ner_pos1, tmp_ner_pos2):
                    tmp_ner_l1[p] = ref_dic['대분류'].loc[tmp_pos_info[0].loc[i]]
                    tmp_ner_l2[p] = ref_dic['중분류'].loc[tmp_pos_info[0].loc[i]]
        ner_sent.append((' ').join(sent_split))
        ner_tag_l1.append((' ').join(tmp_ner_l1))
        ner_tag_l2.append((' ').join(tmp_ner_l2))
    article_sent.extend(ner_sent)
    article_ner_l1.extend(ner_tag_l1)
    article_ner_l2.extend(ner_tag_l2)

In [8]:
label_ner_l1_ = [x for a_ner in article_ner_l1 for x in a_ner.split(' ') if x != 'O' and len(x) > 0 ]
label_ner_l2_ = [x for a_ner in article_ner_l2 for x in a_ner.split(' ') if x != 'O' and len(x) > 0]
tmp_label_ner_l1 = list(set(label_ner_l1_))
tmp_label_ner_l2 = list(set(label_ner_l2_))
tmp_label_ner_l1.sort()
tmp_label_ner_l2.sort()

In [9]:
label_ner_l1 = ['UNK']# + tmp_label_ner_l1
label_ner_l2 = ['UNK']# + tmp_label_ner_l2
for i in tmp_label_ner_l1:
    label_ner_l1.extend([i+'-B', i+'-I'])
for i in tmp_label_ner_l2:
    label_ner_l2.extend([i+'-B', i+'-I'])

In [10]:
article_ner_l1

['O O O O 제품 O O O O 이벤트 O O O O O O 스펙 스펙 O O 제품 O O O O 기술 이벤트 이벤트',
 'O 이벤트 O 제품 O O O O O 제품 O 기술 이벤트 O O O O O O O O O O O O 이벤트 O 이벤트 O O O',
 'O 스펙 제품 O 제품 O O O 제품 O O O 제품 이벤트 O O O 기술 O O O O 기술 O O O O O O O 이벤트',
 '제품 이벤트 O O O O O O O O O O 제품 O O O O 트렌드 O O O O O O O O O 이벤트 O O O O O O O O O O O O 제품 이벤트 O O O O O O O O 제품 스펙 기술 스펙 O O O O O O O O 기술 이벤트 O O 서비스 O O O 스펙 O',
 'O 제품 O O O 이벤트 O O 기술 이벤트 O O O 스펙 기술 서비스 스펙 O O O',
 'O O O 제품 O 이벤트 O O O O O 제품 O O O O O O O O 이벤트',
 'O O O O O O O O 스펙 이벤트 O O O O O O O O O O O O O O',
 'O O 제품 O 트렌드 O 이벤트 제품 제품 O O 스펙 O O O O 이벤트 O O O O',
 'O O 소재 O O 사건/사고 제품 O O O O 제품 제품 O O O O O 스펙 O',
 'O O O O O O O O O O O 제품 O O O O O O O 스펙 O O O',
 'O O 이벤트 O O 제품 O O 제품 스펙 O 스펙 제품 O O O O O 스펙 이벤트 O O 트렌드 O 이벤트',
 'O O O O O O O O O O O O O 스펙 O 소재 이벤트 O 이벤트',
 'O O O O O O O O O 트렌드 O O O O O',
 'O O O 제품 O O 스펙 O O 기술 이벤트 O O O',
 'O O 제품 O 스펙 제품 제품 O O 서비스 O O O O O O 트렌드 O O 이벤트',
 '제품 이벤트 O O O O O O O O O O O O O O O 스

In [11]:
# Train 
'''
with open('LGES_sent_210811.dta', 'wb') as a_sent:
    pickle.dump(article_sent, a_sent)
with open('LGES_sent_ner_level1_210811.dta', 'wb') as a_sent_l1:
    pickle.dump(article_ner_l1, a_sent_l1)   
with open('LGES_sent_ner_level2_210811.dta', 'wb') as a_sent_l2:
    pickle.dump(article_ner_l2, a_sent_l2)  
with open('LGES_label_l1_210811.dta', 'wb') as a_label1:
    pickle.dump(label_ner_l1, a_label1)
'''
# Test 
with open('Test_LGES_sent_210811.dta', 'wb') as a_sent:
    pickle.dump(article_sent, a_sent)
with open('Test_LGES_sent_ner_level1_210811.dta', 'wb') as a_sent_l1:
    pickle.dump(article_ner_l1, a_sent_l1)   
with open('Test_LGES_label_l1_210811.dta', 'wb') as a_label1:
    pickle.dump(label_ner_l1, a_label1)


In [14]:
label_ner_l1

['UNK',
 '기술-B',
 '기술-I',
 '사건/사고-B',
 '사건/사고-I',
 '서비스-B',
 '서비스-I',
 '소재-B',
 '소재-I',
 '스펙-B',
 '스펙-I',
 '이벤트-B',
 '이벤트-I',
 '제품-B',
 '제품-I',
 '트렌드-B',
 '트렌드-I']

In [18]:
print('총 %d 개 문장'%len(article_ner_l1))
article_ner_l1[0]

총 605785 개 문장


'O O O O O O O O O O O O O O O O O O O'

In [19]:
tmp_total_sent = ' '.join(artice_ner_l1)

NameError: name 'artice_ner_l1' is not defined

In [16]:
for k in range(0, db_tab.shape[0]):
    tmp_ = kss.split_sentences(db_tab[5].iloc[k])
    #print(k)
    for l in range(0, len(tmp_)):
        if tmp_[l][:16] == "기아로 서 는 큰 결단 을 내":
            print(k, l)

In [4]:
tmp_ = kss.split_sentences(db_tab[5][1])
print(tmp_[18])

포스코인터내셔널의 곡물 교역량은 2015년 84만 톤에서 지난해 약 800만 톤으로 10배가량 늘었다.


In [14]:
db_tab[5].iloc[24]#, :]

'르노삼성자동차는 2022년형 XM3에 탑재한 인카페이먼트에 CU 편의점 서비스 기능을 16일부터 추가한다고 밝혔다.\n\nXM3 인카페이먼트 편의점 서비스는 전국 1000여개 CU 편의점에서 이용할 수 있다. 차량 안에서 필요한 편의점 판매 상품이 있을 경우 XM3 내비게이션 화면에서 모빌리티 커머스 플랫폼 오윈 애플리케이션(앱)을 통해 주문할 수 있다.\n\n\n\n주문을 실행하면 해당 편의점까지 길 안내를 제공하며 도착 후 점원 호출 기능으로 차량 안에서 상품을 받을 수 있다. 편의점 직원에게 실시간 교통상황을 반영한 고객 도착 예정 시간 정보를 전달, 상품을 미리 준비할 수 있도록 시스템 업데이트도 진행할 예정이다.\n\nXM3 인카페이먼트 편의점 서비스로 구매 가능한 상품은 삼각김밥과 샐러드, 음료 등 간단한 식음료부터 도시락, 생활용품까지 다양하다. 이달의 핫 이슈 상품이나 세트 메뉴, 할인 상품 등도 파악할 수 있다. 편의점별로 구매 가능한 상품은 다를 수 있으며 주류나 담배 구매 서비스는 제공하지 않는다.\n\n\n\n김태준 르노삼성차 영업마케팅 본부장은 “CU 편의점 서비스 기능 추가로 물건을 구매할 때 대면 접촉과 구매 시간을 최소화할 수 있게 됐다”고 말했다.\n\n\n\n정치연기자 '

In [12]:
db_tab

,1,2,4,5
0,2021-07-16,서울경제,2년 넘게 벌여온 배터리 전쟁의 종지부를 찍은 LG와 SK가 이번엔 습식 분리막 시...,viewer 신학철 LG화학 부회장이 14일 열린 온라인 기자 간담회에서 3대 신성...
1,2021-07-16,서울경제,우리나라 경제 성장기에 수출 역군으로 활약했던 종합상사가 환골탈태에 나서고 있다. ...,"< 저작권자 ⓒ 서울경제, 무단 전재 및 재배포 금지 >\n\n우리나라 경제 성장기..."
2,2021-07-16,한국경제,정의선 현대자동차그룹 회장(사진)이 한 달 만에 다시 미국 출장에 나섰다. 미국 시...,귀국길 日들러 양궁대표팀 응원\n\n정의선 현대자동차그룹 회장(사진)이 한 달 만에...
3,2021-07-16,매일경제,세계 전기차 배터리 제조 업체들이 전기차용 배터리 주요 재료인 코발트 가격 상승에 ...,\n\n\n\n세계 전기차·배터리 제조 업체들이 전기차용 배터리 주요 재료인 코발트...
4,2021-07-16,한국경제,르노삼성자동차가 소형 스포츠유틸리티차량(SUV) XM3 2022년형에 탑재된 '인카...,"전국 1000여개 CU편의점서 이용 가능\n\n르노삼성 ""제휴 업체, 범위 늘릴 것..."
...,...,...,...,...
28496,2020-07-16,동아일보,“경찰 예산 삭감하라.” “정의 없이 평화도 없다.” \n \n 미국 독립기념일이던...,지난달 9일 미네소타주 미니애폴리스 시민들이 ‘경찰 예산을 삭감하라’라고 적힌 대형...
28497,2020-07-16,중앙일보,“내 절친이 신종 코로나바이러스 감염증(코로나19)으로 세상을 떠났다. 놀라고 슬프...,친했던 여성 화가 코로나로 숨져\n\n노래에 그리움과 희망 담고 싶어\n\n참여한 ...
28498,2020-07-16,파이낸셜뉴스,21일께 현대차 남양연구소 방문 \n\n \n\n \n이재용 삼성전자 부회장이 다음...,21일께 현대차 남양연구소 방문\n\n\n\n이재용 삼성전자 부회장이 다음 주 현대...
28499,2020-07-16,파이낸셜뉴스,"경북도, GS건설 등 55개 기업과 \n투자양해각서 작년比 22건 늘어 \n\n \...","경북도, GS건설 등 55개 기업과\n\n투자양해각서… 작년比 22건 늘어\n\n\..."
